In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 0.0, 1.0
p_train = np.random.uniform(p_low, p_high, (train_size, 2)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 2)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 2)).astype(np.float32)

## Exact Solver

In [5]:
from src.problem import msQuadratic
model = msQuadratic()

In [6]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("res/qp_exact.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:11<00:00, 13.95it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.301369          0.287015      0.070999
std       0.015466          0.665298      0.029943
min      -0.310000          0.000000      0.051135
25%      -0.310000          0.000000      0.060987
50%      -0.310000          0.000000      0.061791
75%      -0.300000          0.000000      0.062636
max      -0.212920          3.460264      0.186131
Number of infeasible solution: 212


## Heuristic

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    model_rel = model.relax()
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("res/qp_heur.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:13<00:00, 13.56it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.302556          0.338047      0.068560
std       0.015731          0.694741      0.030061
min      -0.310000          0.000000      0.052426
25%      -0.310000          0.000000      0.057103
50%      -0.310000          0.000000      0.058109
75%      -0.300000          0.231860      0.059584
max      -0.212920          3.460264      0.183580
Number of infeasible solution: 335
